##### <b>1. Load custom functions</b>

In [5]:
from functions import *
random_state = 1

##### <b>2. Load data and perform EDA (Exploratory Data Analysis)</b>

In [6]:
survey_df = load_data(file_name="ACME-HappinessSurvey2020.csv")
print(survey_df.head(), "\n")
print(survey_df.info(), "\n")
survey_df.describe()

   Y  X1  X2  X3  X4  X5  X6
0  0   3   3   3   4   2   4
1  0   3   2   3   5   4   3
2  1   5   3   3   3   3   5
3  0   5   4   3   3   3   5
4  0   5   4   3   3   3   5 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Y       126 non-null    int64
 1   X1      126 non-null    int64
 2   X2      126 non-null    int64
 3   X3      126 non-null    int64
 4   X4      126 non-null    int64
 5   X5      126 non-null    int64
 6   X6      126 non-null    int64
dtypes: int64(7)
memory usage: 7.0 KB
None 



,Y,X1,X2,X3,X4,X5,X6
count,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000
mean,0.547619,4.333333,2.531746,3.309524,3.746032,3.650794,4.253968
std,0.499714,0.800000,1.114892,1.023440,0.875776,1.147641,0.809311
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,4.000000,2.000000,3.000000,3.000000,3.000000,4.000000
50%,1.000000,5.000000,3.000000,3.000000,4.000000,4.000000,4.000000
75%,1.000000,5.000000,3.000000,4.000000,4.000000,4.000000,5.000000
max,1.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


The data set is simple - only 126 rows and 6 columns, all integer data type with a range between 0 and 5.<br>
<br>
Since the target variable Y is a categorical variable (0 - unhappy, 1 - happy), convert it accordingly in the data frame.


In [7]:
survey_df["Y"] = survey_df["Y"].astype('category').cat.set_categories([0, 1], ordered=True)

Plot histograms to understand the distribution of each column.

In [8]:
# plot_histograms(data=survey_df,
#                 target="Y", target_figsize=(2,2),
#                 dependent_layout=(2,3), dependent_figsize=(8,6))

For the target variable Y, the distribution is slightly uneven, with more happy customers in the survey data.<br>
<b>To deal with this class imbalance problem, SMOTE (synthetic minority oversampling technique) will be used in the next stages.</b><br>
<br>
The distribution of each dependent variable is also not uniform. However, it would not matter too much as long as the variables/features can provide predictive power.<br>
To better understand these features, perform Chi-square tests.

##### <b>3. Feature selection</b>

In [9]:
# chi_independence_df = run_chi_tests(data=survey_df, target="Y", significance_level=0.05,
#                                     plot_row=2, plot_col=3, figsize=(8,5))
# print("----------------------------------------------------------------------------")
# print("----------------------------------------------------------------------------")
# print("2. Chi-square test of independence")
# print("----------------------------------------------------------------------------")
# print("----------------------------------------------------------------------------")
# print("Table 1. Result of Chi-square test of independence (X1-6 and Y)")
# chi_independence_df.set_index("Independent Variable")

<b>Chi-square test of goodness of fit</b><br>
- This statistical test is often used to evaluate whether or not sample data is representative of the full population.
- The null hypothesis was that there is no significant difference between a variable and its expected frequencies.
- As we failed to reject the null hypothesis for all 6 dependent variables, we can consider that they are representative of the population at a significance level (or alpha) of 0.05.

<b>Chi-square test of independence</b><br>
- This statistical test is used to evaluate whether or not a difference between observed expected data is due to chance. We can consider a relationship between the variables exists when failing to reject the null hypothesis.
- The null hypothesis was that a dependent variable X# and the target variable Y are independent of each other.
- At a significance level of 0.05, we were able to reject the null hypothesis only for X1.
- If we were to tolerate a higher chance of error by increasing the alpha to 0.1, we would be able to reject the null hypothesis for X6 as well, which would mean that X6 and Y are not independent of each other.
- In summary, X2-X5 are independent of Y, meaning they would not be helpful in predicting Y. Whereas X1 would be helpful in predicting Y at a significance level of 0.05 and X6 as well, at a significance level of 0.1.

<b>Relationship between target Y and each dependent variable</b><br>
- X1 and X5 seem to be good features for training a predictive model based on the line charts above - higher X values (dependent variable) generally correspond to higher Y values (target variable). However, the chi-square test of independence failed to reject the null hypothesis of "X5 and Y are independent of each other" when alpha=0.05.
- On the contrary, it appears that it would be hard to predict Y based on the other X variables as fluctuations are observed from the line charts, which were confirmed by the chi-square test of independence.

<b>Summary and next step</b><br>
- <b>X1 (my order was delivered on time) appears to be the most relevant feature to the target Y (customer satisfaction).</b>
- However, to build a more robust model, it would be reasonable to use at least 2 features rather than discarding 5 out of 6 features, especially because the data are not complex nor big.
- X5 and X6 could be useful in training a predictive model but it is unclear at this stage whether using either or both of them would be better.
- <b>As such, perform the chi-square test of independence again</b>, but this time to test whether there is a relationship between X1 and the other dependent variables <b>to determine what features other than X1 can be useful in model training.</b>

In [10]:
# chi_independence_df_X1 = run_chi_tests(data=survey_df.drop(["Y"], axis=1), target="X1", significance_level=0.05,
#                                        plot_row=2, plot_col=3, figsize=(8,5), plot=False,
#                                        goodness_of_fit_test=False)
# print("Table 2. Result of Chi-square test of independence (X1 and X2-6)")
# chi_independence_df_X1.set_index("Independent Variable")

X3, X5 and X6 were found to be not independent of X1, meaning they are related to X1.<br>
<br>
With that, we can now try different combinations of the features (i.e. X1, X3, X5 and X6) to see which combination would result in the best prediction accuracy score.<br>
<br>
Before doing so, evaludate different classifiers to choose the base model for the next steps.

##### <b>4. Model selection</b>

First, split the data into train and test.<br>
<br>
As stated above, SMOTE (synthetic minority oversampling technique) will be used for train and test data separately, to handle the class imbalance problem of target Y.<br>
<br>
Train data will be used for model selection, feature engineering, and hyperparameter tuning.<br>
Test data will only be used at the last step to evaluate the fine-tuned model.<br>

In [11]:
X_train, X_test, y_train, y_test = split_data(
    X=survey_df.drop(["Y"], axis=1),
    y=survey_df["Y"],
    test_size=0.3,
    random_state=random_state,
    oversampling=True)

In [12]:
# eval_models(X=X_train, y=y_train, n_splits=10, n=100, random_state=random_state)

### to be edited
(Tried some of the most common classifiers. RandomForestClassifier sometimes outperformed XGBClassifier but XGBClassifier was so much faster and the performance was as good as RandomForestClassifier.
(add description of XGB classifier)
https://www.nvidia.com/en-us/glossary/data-science/xgboost/)

Use this classifier as our base model for the later steps. Moving forward, try different combinations of the features (X1, X3, X5 and X6) to see which combination would result in the best prediction accuracy score.

In [13]:
# eval_feature_combinations(X=X_train, y=y_train, n_splits=10, n=100, random_state=random_state,
#                         classifier=LogisticRegression(random_state=random_state))

<b>X1 and X3</b> resulted in the best mean prediction accuracy score, thus the other features will not be used from now on.<br>
<br>
Re-define X_train and X_test with the best features combination for the later steps.


In [14]:
X_train_reduced = X_train[["X1", "X3"]]
X_test_reduced = X_test[["X1", "X3"]]

##### <b>5. Feature augmentation</b>

With the three features, try different data augmentation/transformation techniques that will create new features out of the existing features to see whether they improve prediction accuracy.

In [15]:
# eval_transformers(X=X_train_reduced, y=y_train, n_splits=10, n=100, random_state=random_state,
#                 classifier=LogisticRegression(random_state=random_state))

# the best was RBFSampler -> what it is and why it performs well with the data we have?

As we found the mean accuracy scores improved with the transformers, use the best performing transformer for the next steps.

In [16]:
transformer=RBFSampler(random_state=random_state)
X_train_transformed = transformer.fit_transform(X_train_reduced)
X_test_transformed = transformer.transform(X_test_reduced)

##### <b>6. Dimensionality reduction</b>
Now we have more features due to the feature augmentation process, try different dimensionality reduction techniques to see if they help improve accuracy score.

In [17]:
# eval_decomposers(X=X_train_transformed, y=y_train, n_splits=10, n=100, random_state=random_state,
#                 classifier=LogisticRegression(random_state=random_state))

In [18]:
decomposer = KernelPCA(random_state=random_state)
X_train_decomposed = decomposer.fit_transform(X_train_transformed)
X_test_decomposed = decomposer.transform(X_test_transformed)

Decomposers were not helpful in improving accuracy, thus will not be used in the next steps.<br>
<br>
Next is hyperparameter tuning for the classifier.

##### <b>7. Hyperparameter tuning</b>

In [30]:
# n_iter=100
# cv = 2
# classifier = RandomForestClassifier(random_state=random_state)
# params = {
#     "n_estimators": np.arange(10, 200, 10),
#     "criterion": ["gini", "entropy", "log_loss"],
#     "max_depth": [1, 2, 3],
#     "min_samples_split": [2, 3, 4, 5],
#     "min_samples_leaf": [1, 2, 3],
#     "max_features": ["sqrt", "log2", None],
#     "class_weight": ["balanced", "balanced_subsample", None]
# }

# n_iter=10
# cv = 10
# for test_size in [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]:
test_size=0.24
print(f"test_size: {test_size}")
X_train, X_test, y_train, y_test = split_data(
    X=survey_df.drop(["Y"], axis=1),
    y=survey_df["Y"],
    test_size=test_size,
    random_state=random_state,
    oversampling=True)

X_train_reduced = X_train[["X1", "X3"]]
X_test_reduced = X_test[["X1", "X3"]]

transformer=RBFSampler(random_state=random_state)
X_train_transformed = transformer.fit_transform(X_train_reduced)
X_test_transformed = transformer.transform(X_test_reduced)

decomposer = KernelPCA(random_state=random_state)
X_train_decomposed = decomposer.fit_transform(X_train_transformed)
X_test_decomposed = decomposer.transform(X_test_transformed)

for n_iter in np.arange(10, 110, 10):
    # print(f"n_iter: {n_iter}")
    for cv in np.arange(2, 11, 1):
        # print(f"cv: {cv}")
        classifier = LogisticRegression(random_state=random_state)
        params = {
            "penalty": ["l1", "l2", "elasticnet", None],
            "tol": [10.0 ** n for n in np.arange(-10, 0, 1)],
            "C": [10.0 ** n for n in np.arange(-10, 0, 1)],
            "fit_intercept": [True, False],
            "class_weight": ["balanced", None],
            "solver": ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
            "max_iter": np.arange(10, 200, 10),
        }

        best_search = RandomizedSearchCV(estimator=classifier,
                                        param_distributions=params,
                                        scoring="f1",
                                        n_iter=n_iter, # n_iter=10 by default
                                        random_state=random_state,
                                        cv=cv)
        best_search.fit(X_train_decomposed, y_train)
        # print(best_search.best_estimator_, "\n", best_search.best_score_, "\n", best_search.best_params_)

        score = accuracy_score(y_test, best_search.best_estimator_.predict(X_test_decomposed))
        print(f"n_iter: {n_iter}, cv: {cv}, best_score: {round(best_search.best_score_, 2)}, test_score: {round(score, 2)}")
        if score >= 0.73:
            print(f"Test score is higher than 0.70!: {score}\n")
    # print(score, "\n")

test_size: 0.24
n_iter: 10, cv: 2, best_score: 0.67, test_score: 0.47
n_iter: 10, cv: 3, best_score: 0.58, test_score: 0.71
n_iter: 10, cv: 4, best_score: 0.57, test_score: 0.71
n_iter: 10, cv: 5, best_score: 0.55, test_score: 0.71
n_iter: 10, cv: 6, best_score: 0.55, test_score: 0.71
n_iter: 10, cv: 7, best_score: 0.58, test_score: 0.71
n_iter: 10, cv: 8, best_score: 0.56, test_score: 0.71
n_iter: 10, cv: 9, best_score: 0.56, test_score: 0.71
n_iter: 10, cv: 10, best_score: 0.58, test_score: 0.71
n_iter: 20, cv: 2, best_score: 0.67, test_score: 0.47
n_iter: 20, cv: 3, best_score: 0.61, test_score: 0.65
n_iter: 20, cv: 4, best_score: 0.67, test_score: 0.5
n_iter: 20, cv: 5, best_score: 0.66, test_score: 0.65
n_iter: 20, cv: 6, best_score: 0.57, test_score: 0.65
n_iter: 20, cv: 7, best_score: 0.58, test_score: 0.71
n_iter: 20, cv: 8, best_score: 0.56, test_score: 0.71
n_iter: 20, cv: 9, best_score: 0.56, test_score: 0.71
n_iter: 20, cv: 10, best_score: 0.58, test_score: 0.71
n_iter: 30,

##### <b>8. Evaluate the tuned model using test data</b>

In [20]:
# score_dict = {}
# for random_state in range(100):
#     classifier = RandomForestClassifier(random_state=random_state, **best_search.best_params_)
#     transformer = RBFSampler(random_state=random_state)
#     decomposer = KernelPCA(random_state=random_state)
#     clf = make_pipeline(transformer, decomposer, classifier)
#     clf.fit(X_train_reduced, y_train)
#     score = accuracy_score(y_test, clf.predict(X_test_reduced))
#     score_dict[random_state] = score

# score_dict_sorted = dict(sorted(score_dict.items(), key=lambda x: x[1], reverse=True))
# print(score_dict_sorted[1])
# score_dict_sorted

##### <b>9. Save the best performing model for future use</b>

##### <b>10. Conclusion</b>

learning points, insights, future work, etc.


- changed n_splits from 5 to 2 since the data set is so small that the accuracy prediction could not be good when the split was too granular.